In [1]:
import time
import pandas as pd
import numpy as np

In [2]:
from __future__ import print_function
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('data/dataframe.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
df.shape

(9092, 19)

In [4]:
df.head()

,tweet,product_,emotion,lemmatizer_tweets,textblob_polarity,textblob_subjectivity,vs_neg,vs_neu,vs_pos,vs_compound,nrc_sentiment,gi_sentiment,henry_sentiment,huliu_sentiment,jockers_sentiment,lm_sentiment,senticnet_sentiment,sentiword_sentiment,socal_sentiment
0,"['wesley83', 'have', '3G', 'iPhone', '3', 'hrs...",iPhone,0,wesley83 have 3G iPhone 3 hr tweeting RISE Aus...,-0.200000,0.400000,0.223,0.777,0.000,-0.6486,0.0,-0.333333,0.0,-1.0,-1.000000,0.0,-0.09520,-0.221875,-1.192154
1,"['jessedee', 'Know', 'fludapp', 'Awesome', 'iP...",iPad or iPhone App,1,jessedee Know fludapp Awesome iPad iPhone app ...,0.466667,0.933333,0.000,0.528,0.472,0.9100,1.0,1.000000,0.0,1.0,0.416667,0.0,0.47500,0.175000,2.177190
2,"['swonderlin', 'not', 'wait', 'iPad', '2', 'al...",iPad,1,swonderlin not wait iPad 2 also should sale do...,-0.155556,0.288889,0.000,1.000,0.000,0.0000,-1.0,-1.000000,-1.0,-1.0,-0.625000,-1.0,-0.30550,-0.289062,-1.000000
3,"['sxsw', 'hope', 'year', 'festival', 't', 'cra...",iPad or iPhone App,0,sxsw hope year festival t crashy this year iPh...,0.000000,0.000000,0.000,0.596,0.404,0.7269,1.0,1.000000,0.0,0.0,0.500000,0.0,0.07160,0.250000,2.841547
4,"['sxtxstate', 'great', 'stuff', 'Fri', 'SXSW',...",Google,1,sxtxstate great stuff Fri SXSW Marissa Mayer G...,0.800000,0.750000,0.000,0.796,0.204,0.6249,0.0,1.000000,0.0,1.0,0.500000,1.0,0.55125,0.083333,1.554026


In [5]:
df['emotion'] = np.where(df['emotion'] == 4, 2, df['emotion'])

In [6]:
data = df.drop(columns=['emotion', 'tweet', 'product_', 'lemmatizer_tweets'])
target = df['emotion']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.20, random_state=1)


In [8]:
# sm = SMOTE( random_state=23)
# X_train, y_train = sm.fit_sample(X_train, y_train)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7273, 15)
(1819, 15)
(7273,)
(1819,)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.20, random_state=1)

# tl = TomekLinks()
# X_train, y_train = tl.fit_resample(X_train, y_train)

batch_size = 8 # how many folds to separate data
num_classes = 4 # how many classes in outcomes
epochs = 15

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = y_train.astype('uint8')
y_test = y_test.astype('uint8')


# X_train = X_train.reshape(1738, 15)
# X_test = X_test.reshape(1819, 15)
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')



# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# specifying the model structure
model = Sequential()

# specify the first hidden layer
model.add(Dense(100, activation='relu', input_shape=(15,)))
model.add(Dropout(0.2))

# specify the second layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

# # specify the third layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

# # specify the fourth layer
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.2))

# # specify the 5th layer
# model.add(Dense(50, activation='relu'))

# specify the output layer
model.add(Dense(num_classes, activation='softmax')) # switched linear to sofmax

model.summary()


model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1600      
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 4

In [11]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8828907012939453
Test accuracy: 0.6107751727104187


In [12]:
from ann_visualizer.visualize import ann_viz;


In [16]:

ann_viz(model, title="Sentiment Neural Net")